# Python fitVirusXX
## Version 10.X.1
## Date Updated: 01.25.2021
## Author: Azmyin Md. Kamal

# Version info
* Calculates logisitc fit for week level data for first virus wave only
* funcsXX.py must be present in the same directory as this notebook
* This version natively supports coloring each 'phase length' similar to Dr. Batista's paper https://doi.org/10.1101/2020.02.16.20023606. To activate this feature head over to funcsXX.py and change plot_cumu_phases() the code for color commented as HARDCODED

In [ ]:
# Load all modules, function definitions and important variable definitions
from funcsXX import *

df = pd.read_csv("Cases_All.csv", header = 0)
#df.head(10)

In [ ]:
# Reproduce Figure 1, example of analyzing only one state
state_names = ['Arizona'] # If you want to fit data for only one state

# From the paper
#state_names = ['Idaho', 'Ohio','Oregon', 'Texas', 'Nevada', 'Hawaii', 'Georgia', 'Florida', 'Alabama', 'Tennessee', 'North Carolina', 'Arizona', 'California', 'Mississippi', 'Iowa', 'Louisiana', 'South Carolina', 'Kentucky', 'New Mexico', 'Washington']

In [ ]:
# Keep week array in memory
week_array_file = df['week'].values
#print(week_array_file)
#print(np.shape(week_array_file))
const_add = week_array_file[0] - 1
print(const_add)

In [ ]:
loss_state = []
for state_now in state_names:
    # Main function call
    b_wave1, timestamp_week, week_agg, y_pred, rr2_wave1 = master_fn(df = df,state_name = state_now, const_add = const_add, debug = False, plot_res = False)
    
    bb_fit1 = b_wave1
    t_cumu = timestamp_week
    y_act = week_agg
    y_pred = logisticFun(t_cumu, bb_fit1) # Calculate Prevelance
    #print(y_pred)

    # Calculate single wave incidence
    I_pred = fit_incidence_single(y_act,t_cumu,bb_fit1)
    I_act_w1 = np.diff(y_act)
    I_act_w1 = np.insert(I_act_w1, [0], [I_act_w1[0]])
    max_w1 = max(I_pred)

    # Find phase location
    w1_loc_ls1 = fit_single_phase(bb_fit1, t_cumu, False)
    # print(f"Phase location before scaling --> {w1_loc_ls1}\n") # Debug
    max_end = max(timestamp_week)

    # Scale to right scale
    w1_loc_ls = np.asarray(scale_to_dat_week(w1_loc_ls1, const_add, max_w1))
    # print(f"Phase location after scaling --> {w1_loc_ls}\n") # Debug

    # Setup variables for plotting    
    w2_loc_ls = [0,0,0,0,0]
    t_span = week_array_file

    # Plot cumulative phases
    fontSIZE = 15
    plt.rcParams.update({'font.size': fontSIZE})
    plot_cumu_phases(const_add, w1_loc_ls1,bb_fit1, t_span, y_act, y_pred, I_act_w1, fontSIZE)